In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt

/Users/kinda/anaconda3/envs/ai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
data_dir = '~/Desktop/pyTorch-learn'

# 定义转换
transformations = transforms.Compose([
    transforms.Resize((64, 64)),  # 缩放所有图像到相同的大小
    transforms.ToTensor(),         # 转换为PyTorch张量
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 标准化图像
])

In [28]:
# 加载数据集
train_dataset = datasets.ImageFolder(data_dir + '/train', transform=transformations)
test_dataset = datasets.ImageFolder(data_dir + '/test', transform=transformations)

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [57]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3).float()
        self.pool = nn.MaxPool2d(2, 2).float()
        self.conv2 = nn.Conv2d(6, 16, 3).float()
        self.fc1 = nn.Linear(16 * 13 * 13, 120).float()
        self.fc2 = nn.Linear(120, 84).float()
        self.fc3 = nn.Linear(84, 2).float()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 13 * 13)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

In [58]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [59]:
# set device to MPS
device = torch.device("mps")

In [60]:
for epoch in range(10):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

[1,   200] loss: 0.693


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
# # 测试模型
# test_loss = 0.0
# class_correct = list(0. for i in range(2))
# class_total = list(0. for i in range(2))

# model.eval()
# for inputs, labels in test_loader:
#     # 将输入和标签放入GPU
#     inputs = inputs.to('mps')
#     labels = labels.to('mps')

#     # 前向传递
#     outputs = model(inputs)
#     loss = criterion(outputs, labels)

#     # 更新测试损失
#     test_loss += loss.item()*inputs.size(0)

#     # 获取预测结果
#     _, predicted = torch.max(outputs, 1)
#     correct = np.squeeze(predicted.eq(labels.data.view_as(predicted)))

#     # 更新分类精度
#     for i in range(len(labels)):
#         label = labels.data[i]
#         class_correct[label] += correct[i].item()
#         class_total[label] += 1

# # 计算测试结果
# test_loss = test_loss/len(test_loader.dataset)
# test_acc = 100.0*np.sum(class_correct)/np.sum(class_total)

# print('Test Loss: {:.6f}\n'.format(test_loss))
# print('Test Accuracy: {:.2f}% ({}/{})'.format(test_acc, np.sum(class_correct), np.sum(class_total)))


In [ ]:
# 绘制图表
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Test loss')
plt.legend(frameon=False)
plt.show()

plt.plot(test_accs, label='Test accuracy')
plt.legend(frameon=False)
plt.show()
